## CSV FILES

In [ ]:
import csv
import random

file_name = "notas_arqui.csv"

In [ ]:
def escribir_csv(cant_filas):
    contenido = f"codigo,{','.join(f'lab{idx + 1}' for idx in range(12))},e1,e2\n"
    codigo_base = 20211899

    for i in range(cant_filas):
        fila = f"{codigo_base + i},{','.join(str(random.randint(0,20)) for idx in range(14))}\n"
        contenido += fila

    contenido = contenido[:-1] # Quita el último salto de línea

    with open(file_name, "w+") as f:
        f.write(contenido)

In [ ]:
def leer_csv():
    with open(file_name, "r") as f:
        contenido = f.read()

    datos = contenido.split("\n")

    return datos

In [ ]:
def leer_csv_dict():
    with open(file_name, mode = 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        return list(csv_reader)

In [ ]:
escribir_csv(2)
datos = leer_csv()
datos_dict = leer_csv_dict()

print(datos)
print(datos_dict)  

# Para leer los datos específicos del datos_dict

print([row['codigo'] for row in datos_dict])  # Lista de códigos

## sockets

In [ ]:
import socket

SOCK_BUFFER = 1024
HOST = "0.0.0.0"
PORT = 5000

- Servidor

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # Son datos por default, se copia y pega
server_address = (HOST, PORT)

print(f"Levantando el servidor en {server_address[0]}, con puerto {server_address[1]}")

sock.bind(server_address)

sock.listen(5) #cuantos clientes van a escuchar

while True:
    print("Esperando conexiones...")
    conn, client_address = sock.accept() #mi codigo no va a avanzar mas de esta linea hasta que el cliente acceda
    print(f"Conexion de {client_address[0]}:{client_address[1]}")

    try:
        while True:
            dato = conn.recv(SOCK_BUFFER)

            if dato:
                print(f"Recibí: {dato}")
                conn.sendall(dato)
            else:
                print(f"No hay más datos")
                break
    except ConnectionResetError: #ecept exception: | eso chapa todas las excepciones, pero no es recomendable. Son constantes, no etiquetas.
        print("El cliente cerró la conexión de manera abrubta")
    finally:
        print("Cerrando la conexión")
        conn.close()

- Cliente

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_address = (HOST, PORT)

print(f"Conectandonos al servidor en {server_address[0]} en el puerto {server_address[1]}")

sock.connect(server_address)

msg = "hola mundo!"

sock.sendall(msg.encode("utf-8"))
dato = sock.recv(SOCK_BUFFER)

sock.close()

print(f"Recibí: {dato}")

- Try except ejemplo:

In [ ]:
a = 1
b = '2'

try:
    c = a + b
except TypeError:
    print("Ocurrió un error de tipo")
    if not isinstance(a, int):
        a = int(a)

    if not isinstance(b, int):
        b = int(b)

    c = a + b
finally:
    print(f"el resultado de a + b es {c}")

- JSON

In [ ]:
import json

data = {
    "nombre": "Juan",
    "edad": 30,
    "casado": True,
    "hijos": ["Ana", "Luis"],
    "direccion": {
        "calle": "123 Calle Principal",
        "ciudad": "Ciudad Ejemplo"
    }
}

json_string = json.dumps(data)  # Convierte el diccionario a una cadena JSON
print(json_string)

data_2 = json.loads(json_string)  # Convierte la cadena JSON a un diccionario
print(data_2)

# Para enviar como socket hacemos lo siguiente
sock.sendall(json_string.encode('utf-8'))

## Concurrencia

### Async IO

In [ ]:
import asyncio
import time

# 2 palabras clave: async y await

In [ ]:
async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

if __name__ == "__main__":
    s = time.perf_counter()
    asyncio.run(main())
    elapsed = time.perf_counter() - s
    print(f"{__file__} executed in {elapsed:0.2f} seconds.")

Servidor con Asyncio

In [ ]:
import asyncio
import json

SOCK_BUFFER = 1024
HOST = "0.0.0.0"
PORT = 5000

In [ ]:
async def handle_client(reader, writer):
    print("Cliente conectado")

    msg_bytes = await reader.read(SOCK_BUFFER)
    print(f"Recibi: {msg_bytes}")
    if msg_bytes:
        msg_respuesta = 'Hola mundo'
        msg_respuesta_str = json.dumps(msg_respuesta)

        await writer.write(msg_respuesta_str.encode("utf-8"))
        await writer.drain()
    
    writer.close()
    await writer.wait_closed()
    print("conexion cerrada")

async def main():
    server_address = (HOST, PORT)

    server = await asyncio.start_server(handle_client, server_address[0], server_address[1])

    async with server:
        print("Empezando servidor...")
        await server.serve_forever()


if __name__ == '__main__':
    asyncio.run(main())

Cliente con Asyncio

In [55]:
import asyncio
import json

SOCK_BUFFER = 1024
HOST = "0.0.0.0"
PORT = 5000

In [ ]:
async def send(writer, msg):
    print(f"Enviando mensaje: {msg}")
    msg_bytes = msg.encode("utf-8")
    writer.write(msg_bytes)
    await writer.drain()


async def recv(reader):
    print("Esperando respuesta")
    result_bytes = await reader.read(SOCK_BUFFER)
    print(f"recibi en bytes {result_bytes}")
    result = result_bytes.decode("utf-8")

    return result


async def client(server_ip: str, server_port: int, msg: str):
    server_address = (server_ip, server_port)
    print(f"Conectando a {server_address[0]}:{server_address[1]}")
    reader, writer = await asyncio.open_connection(server_address[0], server_address[1])
    print("Conectado")
    await send(writer, msg)
    
    res = await recv(reader)

    print("cerrando la conexion")
    writer.close()
    await writer.wait_closed()

    return res

### Threading

### Multiprocesing